In [2]:
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import os

In [4]:
# Set up data paths
train_dir = 'data/train/'
test_dir = 'data/test/'

In [5]:
# Image data generator for real-time data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.3,  # Increase shift range
    height_shift_range=0.3,
    shear_range=0.3,  # Increase shear range
    zoom_range=0.3,  # Increase zoom range
    horizontal_flip=True,
    vertical_flip=True,  # Add vertical flip
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [6]:
# Create generators for training and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to a common size
    batch_size=32,
    class_mode='categorical'  # Since it's a multiclass classification
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 1211 images belonging to 4 classes.
Found 98 images belonging to 4 classes.


In [8]:
# Generate the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.6),  # Dropout for regularization
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for multiclass
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [9]:
%pip install scipy



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=200,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 24s 626ms/step - accuracy: 0.3586 - loss: 1.4649 - val_accuracy: 0.5000 - val_loss: 1.2123
Epoch 2/200
 1/37 ━━━━━━━━━━━━━━━━━━━━ 7s 206ms/step - accuracy: 0.5000 - loss: 1.1201

c:\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5000 - loss: 1.1201 - val_accuracy: 0.5312 - val_loss: 1.2223
Epoch 3/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 336ms/step - accuracy: 0.4879 - loss: 1.1601 - val_accuracy: 0.5208 - val_loss: 1.1852
Epoch 4/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5625 - loss: 1.0398 - val_accuracy: 0.5104 - val_loss: 1.2017
Epoch 5/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 14s 375ms/step - accuracy: 0.5430 - loss: 1.1115 - val_accuracy: 0.5312 - val_loss: 1.3833
Epoch 6/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5625 - loss: 1.1478 - val_accuracy: 0.5521 - val_loss: 1.3596
Epoch 7/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 324ms/step - accuracy: 0.5853 - loss: 1.0343 - val_accuracy: 0.5938 - val_loss: 1.1617
Epoch 8/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6562 - loss: 0.8887 - val_accuracy: 0.5833 - val_loss: 1.1776
Epoch 9/200
37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 331ms/step - accuracy: 0.5705 - loss: 1.0377 - val_accuracy: 0.5312 -

In [13]:
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7759 - loss: 0.7580 
Test accuracy: 0.7653061151504517


In [80]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_snake(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions[0])
    class_label = list(train_generator.class_indices.keys())[class_idx]
    return class_label

# Example usage
result = predict_snake(r'data\test\download.jpeg')
print(f'The snake species is: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
The snake species is: Bungarus multicinctus
